## Google Earth Engine (GEE)


GEE is a cloud-based platform for massive-scale analysis of geospatial data. GEE allows users access to a petabyte-scale archive of publicly available remotely sensed imagery, ancillary data and computational tools to accomplish a myriad of remote sensing and geospatial tasks at unprecedented speeds and scales. GEE is free for non-commercial use provided users sign up for a GEE account.

## Earth Engine APIs

Earth Engine is available through  JavaScript and Python Application Program Interfaces (APIs). The JavaScript API is accessible via a web-based Integrated Development Environment (IDE) called the Code Editor. Through  this platform users can write and execute scripts to share and repeat geospatial analysis and processing workflows. To access Code Editor one can navigate to [https://code.earthengine.google.com/](https://code.earthengine.google.com/) on Google Chrome web browser. The Python API alternative to Earth Engine Javascript API is Google Colaboratory.

## Google Colaboratory 'Colab'

Colaboratory, or "Colab" for short, allows you to write and execute Python directly in your browser, with Zero configuration required, Free access to GPUs, and Easy sharing. The [Welcome to Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb?utm_source=scs-index#scrollTo=GJBs_flRovLc) document has all the necessary info to get started.


### Accounts

#### Google:

To get started with Colab, you will need a [Google Account](https://accounts.google.com/signup/v2/webcreateaccount?flowName=GlifWebSignIn&flowEntry=SignUp). This can be @gmail or an institution/custom google account. You will use this account to access Google Drive, Google Colab, and activate Google Earth Engine.
To use the ONS google account, you will require a Google Cloud Titan security key to access the EE code editor. One can obtain this key through ONS Service Desk. 


#### Google Earth Engine:

You wil also need an account for Google Earth Engine. You can [sign up here](https://accounts.google.com/ServiceLogin/webreauth?service=ah&passive=true&continue=https%3A%2F%2Fuc.appengine.google.com%2F_ah%2Fconflogin%3Fcontinue%3Dhttps%3A%2F%2Fsignup.earthengine.google.com%2F&flowName=GlifWebSignIn&flowEntry=ServiceLogin), using your Google Account from the previous step to link everything together nicely. 





## Connections

#### Connect Google Drive:

You can access files in Drive in a number of ways, including:

- Mounting your Google Drive in the runtime's virtual machine
- Using a wrapper around the API such as [PyDrive](https://pythonhosted.org/PyDrive/)
- Using the [native REST API](https://developers.google.com/drive/api/v3/about-sdk)

The example below shows how to mount your Google Drive on your runtime using an authorization code. 
Run the code block below. The output will show a link you have to open, copy the code from the page that loads and paste back into a cell provided.

In [ ]:
### Mount Google Drive

# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

**Explore your drive**    

Now that you've connected (mounted) google drive to your session, you can access it as a local disk.

Explore using python to list files:

In [ ]:

    
import os

# list the folders and files in your drive
mydrive = '/content/gdrive/My Drive'
colab = '/content/gdrive/My Drive/Colab Notebooks'
print(os.listdir(mydrive))

# list the current working directory
print(os.getcwd())

# # create a sub-folder called data under your 'Colab Notebooks'
data_folder = os.path.join(colab,'data')
os.path.isdir(data_folder)
if (not os.path.isdir(data_folder)):
  os.mkdir(data_folder)
# list the contents
print(os.listdir(colab))

# root directory for outputs is set to your google drive
my_root_dir = "/content/gdrive/My Drive/Colab Notebooks/data"

### Import, Authenticate and Initialize Earth Engine API

The Earth Engine API (ee) is installed by default in Google Colab so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.



In [ ]:
# If not on Colab you'll need to install the earth-engine Python API
!pip install earthengine-api #earth-engine Python API

# import Python API. 
import ee

# Athenticate to EE servers. 
ee.Authenticate()

# Initialize the API
ee.Initialize()


#The authorization workflow will generate a code, which you should paste in the box below. 

### Install packages and libraries

In [ ]:
# Install packages
%pip install geopandas rasterio rasterstats shapely
%pip install folium 

#### **Import libraries**

In [ ]:
# import required libraries 

from os import path as op
import pickle

import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.features import rasterize
from rasterstats.io import bounds_window
import rasterstats
import folium

### Do a search
Let's query Earth Engine [data collections] by looking for Imagery of UK.

We are going to:

Define a bounding box or center point  using [geojson.io](https://geojson.io/#map=6/54.489/-3.922)     
Identify the collection   
Query and create an ImageCollection   
Display on a map   
Export   ?     
Reload Raster and Explore  ? 

In [ ]:
# Define  bounding box using 

# Make sure we have the libraries we need
# %pip install folium
# %pip install geopandas

In [ ]:
# Paste the Geojson from geojsonio
aoi_geojson = '''{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -2.0654296875,
              61.14323525084058
            ],
            [
              -5.185546875,
              60.34869562531862
            ],
            [
              -7.9541015625,
              58.37867853932655
            ],
            [
              -10.107421874999998,
              54.39335222384589
            ],
            [
              -6.3720703125,
              53.461890432859114
            ],
            [
              -6.943359374999999,
              49.781264058178344
            ],
            [
              -2.197265625,
              50.00773901463687
            ],
            [
              1.9335937499999998,
              51.17934297928927
            ],
            [
              2.5927734375,
              52.82932091031373
            ],
            [
              -0.48339843749999994,
              55.99838095535963
            ],
            [
              -1.3623046875,
              58.65408464530598
            ],
            [
              0.52734375,
              61.33353967329144
            ],
            [
              -2.0654296875,
              61.14323525084058
            ]
          ]
        ]
      }
    }
  ]
}'''

# Now let's read the geojson into a GeoPandasDataframe
aoi = gpd.read_file(aoi_geojson)
print(aoi) # so we can see what it looks like

#Get the bounding box
bbox = aoi.total_bounds
print(bbox) # see the coordinates

#Make it a GEE rectangle
gee_aoi = ee.Geometry.Rectangle(bbox.tolist())
center = aoi.centroid[0]

In [ ]:
# load map

uk_osm = folium.Map(location=[center.y, center.x], tiles="OpenStreetMap", zoom_start=6)

folium.Marker(
    location=[center.y, center.x],
    popup='Center',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(uk_osm)

folium.features.GeoJson(aoi_geojson,
                        style_function = lambda x: {'color':'blue', 'fillcolor':'transparent'}
                        ).add_to(uk_osm)
    
uk_osm

### Query Google Earth Engine
Now lets pick a collection of imagery to query for the region. We can choose from anything in the Google Earth Engine [data catalog](https://developers.google.com/earth-engine/datasets/catalog)

Let's start with the [Landsat 8 Surface Reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2) from 2020, using a cloud mask and selecting the bands to make an R,G,B mosiac

In [ ]:
# To make a map we first need some helper functions

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

# map and display 
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
# Fetch Landsat imagery from GEE and make a map

l8_image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterDate('2020-04-01', '2020-08-31')\
    .median() \
    

# image visible parameters for making map
l8_vis_params = {
  'bands': ['B4', 'B3', 'B2'], # true color visible bands
  'min': 0,
  'max': 3000,
}

# make and display map
Mapdisplay(center=[center.y, center.x],
           dicc={'L8':l8_image.getMapId(l8_vis_params)}, 
           zoom_start=5)

#### Sub-setting and exporting to google Drive

At some point we often want to export data from Earth Engine for other uses. You may export because you need access to algorithms, data, or map making tools that just aren't possible in Earth Engine.


In [ ]:
# Subset to our aoi
band_sel = ('B2', 'B3', 'B4', 'B5')

l8_image_aoi = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterDate('2020-01-01', '2020-08-31')\
    .select(band_sel)\
    .filterBounds(gee_aoi)\
    .median()

l8_image_aoi.getInfo()

Mapdisplay(center=[center.y, center.x],
           dicc={'L8':l8_image_aoi.getMapId(l8_vis_params)}, 
           zoom_start=5)

## Calculate NDVI

In [ ]:
# calculate NDVI using NDVI built-in function
def getNDVI(image):
  return image.normalizedDifference(['B4', 'B3'])
image1 = ee.Image(l8_image_aoi) # l8_image_aoi is the landsat image

# compute NDVI from scene
l8_ndvi = getNDVI(image1)
ndviParams = {'palette': ['#0000cd', '#006400', 'ce7e45', '#ff0000', '22cc04', '207401', '012e01']}

# make and display ndvi map
Mapdisplay(center=[center.y, center.x],
           dicc={'L8':l8_ndvi.getMapId(ndviParams)}, 
           zoom_start=5)

### Display composites from NOAA CDR AVHRR NDVI

In [ ]:
# Fetch NOAA CDR AVHRR NDVI: Normalized Difference Vegetation Index, Version 5 composites  from GEE and make a map

ndvi_image = ee.ImageCollection('NOAA/CDR/AVHRR/NDVI/V5')\
      .filter(ee.Filter.date('2018-05-01', '2018-06-01'));

ndvi_vis = {
  'bands': ['NDVI'],
  'min': -1000, 'max': 5000,
  'palette': ['ffffff', 'ce7e45', 'fcd163', 'c6ca02', '22cc04', '99b718', '207401', '012e01']
}
                  
# make and display map
Mapdisplay(center=[center.y, center.x],
           dicc={'NDVI':ndvi_image.getMapId(ndvi_vis)}, 
           zoom_start=5)